In [ ]:
import requests
import pandas as pd
from datetime import datetime

def get_future_weather_forecast(
    latitude: float,
    longitude: float,
    start_date: str, # format: 'YYYY-MM-DD'
    end_date: str,   # format: 'YYYY-MM-DD'
    hourly_variables: list = None,
    daily_variables: list = None,
    timezone: str = "auto",
    model: str = "gfs_seamless" # default model
) -> dict:
    """
    retrieve future weather forecasts based on the specified latitude, longitude, time range, and weather variables.

    Args:
        latitude (float): Latitude of the location to query.
        longitude (float): Longitude of the location to query.
        start_date (str): forecast start date in 'YYYY-MM-DD' format
        end_date (str): forecast end date in 'YYYY-MM-DD' format
        hourly_variables (list, optional): list of hourly weather variables to retrieve
                                          if None, the default full list of hourly variables will be used
        daily_variables (list, optional): list of daily weather variables to retrieve
                                        if None, the default full list of daily variables will be used
        timezone (str, optional): time zone setting. Default is "auto".
        model (str, optional): forecast model to use. Default is "gfs_seamless"

    Returns:
        dict: dictionary containing the weather forecast data. Returns None if the request fails
    """
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "timezone": timezone,
        "models": model,
    }

    if hourly_variables is None:
        hourly_variables = [
            "rain", "precipitation_probability", "precipitation", "apparent_temperature",
            "relative_humidity_2m", "temperature_2m", "weather_code", "cloud_cover",
            "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility",
            "wind_speed_10m", "temperature_80m", "temperature_120m", "temperature_180m",
            "showers", "pressure_msl", "surface_pressure", "wind_speed_80m",
            "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m",
            "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m", "evapotranspiration",
            "et0_fao_evapotranspiration", "vapour_pressure_deficit", "dew_point_2m",
            # "terrestrial_radiation_instant" # this variable may not be directly available in the forecast API and has been removed
        ]
    
    if daily_variables is None:
        daily_variables = [
            "weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max",
            "apparent_temperature_min", "sunrise", "daylight_duration", "sunset", "sunshine_duration",
            "rain_sum", "showers_sum", "precipitation_sum", "precipitation_hours",
            "precipitation_probability_max", "uv_index_max", "wind_speed_10m_max",
            "wind_gusts_10m_max"
        ]

    params["hourly"] = ",".join(hourly_variables)
    params["daily"] = ",".join(daily_variables)

    try:
        print(f"fetching weather forecast from {start_date} to {end_date} for location ({latitude}, {longitude})")
        response = requests.get(base_url, params=params, timeout=15)
        response.raise_for_status() 
        data = response.json()
        print(f"successfully retrieved forecast data")
        return data
    except requests.exceptions.RequestException as e:
        print(f"failed to retrieve forecast data ({latitude}, {longitude}) - {e}")
        return None
    except Exception as e:
        print(f"an error occurred while processing the forecast data ({latitude}, {longitude}) - {e}")
        return None

# --- example ---

target_lat = 22.997
target_lon = 120.213

forecast_start = "2025-06-01"
forecast_end = "2025-06-03"

forecast_data = get_future_weather_forecast(
    latitude=target_lat,
    longitude=target_lon,
    start_date=forecast_start,
    end_date=forecast_end,
    hourly_variables=None, # allow the function to use the default full list of hourly variables.
    daily_variables=None   # allow the function to use the default full list of daily variables.
)

if forecast_data:
    # process hourly forecast data
    if 'hourly' in forecast_data:
        print("\n--- hourly forecast ---")
        hourly_df = pd.DataFrame(forecast_data['hourly'])
        hourly_df['time'] = pd.to_datetime(hourly_df['time'])
        # display all columns to help verifying the retrieved variables
        pd.set_option('display.max_columns', None) # set display to show all columns
        print(hourly_df.head())
        # hourly_df
        print(f"a total of {len(hourly_df)} hourly records")

    # process daily forecast data
    if 'daily' in forecast_data:
        print("\n--- daily forecast ---")
        daily_df = pd.DataFrame(forecast_data['daily'])
        daily_df['time'] = pd.to_datetime(daily_df['time'])
        pd.set_option('display.max_columns', None) # set display to show all columns
        print(daily_df)
        # daily_df
else:
    print("failed to retrieve the weather forecast")